In [130]:
import instaloader
import os
import datetime
from itertools import takewhile, dropwhile
import pandas as pd
from glob import glob
import orjson
from src.yaml_loader import yaml_loader

In [153]:
insta_vars = yaml_loader("variables.yaml")
USER = insta_vars["user"]
PASSWORD = insta_vars["pswd"]
list_of_users = insta_vars["username"]

# daterange of scraping
startdate = datetime.datetime(2021,1,1)
enddate = datetime.datetime(2022,8,1)
# path parent folder
initial_path = insta_vars["initial_path"]

data:
- Accoount Name
- User name
- Followers at posting
- post created
- type posting
- Likes
- Comments
- URL POst
- Photo URL
- Description

In [135]:
def get_instagram_posts(user_login, password, username : list, startdate, enddate, save_folder='data'):
   # Create an instaloader object with parameters
   L = instaloader.Instaloader(download_pictures = False, download_videos = False, download_comments= False, compress_json = False)
   
   # Log in with the instaloader object
   L.login(user_login , password)
   # Search the instagram profile
   profile = instaloader.Profile.from_username(L.context, username)
   # Scrape the posts
   posts = profile.get_posts()
   os.chdir(os.path.join(initial_path, save_folder))
   for post in takewhile(lambda p: p.date > startdate, dropwhile(      lambda p : p.date > enddate, posts)): 
    print(post.date)
    L.download_post(post, target = profile.username)
   os.chdir(initial_path)

In [138]:
# Call the function. This will automatically store all the scrape data in a folder in your current working directory
for user in list_of_users:
  get_instagram_posts(USER, PASSWORD,user, startdate, enddate)

2022-07-02 11:48:20
[menjadi akhir untuk awal menj…] json 
2021-08-13 12:00:49
[finally, paket lengkap.  [sli…] json 
2021-07-22 09:00:38
[firstreel.  @dinapakaya] json 
2022-09-19 12:27:13
[Rumah Kita telah selesai. tdk…] json 
2022-09-13 13:35:22
[better together.] json 
2022-08-25 10:00:36
[🌋] json 
2022-08-21 16:18:26
[the winner of Cultural Immers…] json 
2022-08-19 15:01:55
[menguat, menggema 💫 Apresiasi…] json 
2022-08-14 11:22:25
[siap-siap berlayar teman-tema…] json 
2022-08-01 12:28:40
[selain pacar, aku selalu anda…] json 
2022-07-22 15:03:26
[join to the club! @apertura.s…] json 
2022-07-21 08:26:03
[sat..set..sat..set..🏸🏸 super …] json 
2022-07-10 11:34:11
[she gets prettier every time …] json 
2022-07-08 09:03:26
[Jalipata wa, hadir dan meriah…] json 
2022-06-18 14:09:26
[petualangan dan pengalaman ba…] json 
2022-06-08 14:58:23
[bianca day’s] json 
2022-06-07 09:19:20
[GIVEAWAY TIME!!  yg mau dapet…] json 
2022-06-04 14:55:49
[not the result we wanted but …] json 
2022-0

In [174]:
def parse_instafiles(username, path):
    """ 
    This function loads in all the json files generated by the      instaloader package and parses it into a csv file.
    """
    #print('Entering provided directory...')
    os.chdir(os.path.join(path+"/data/", username))
    
    columns = ['filename', 'datetime', 'type', 'locations_id', 'locations_name', 'mentions', 'hashtags', 'video_duration']
    
    dataframe = pd.DataFrame(columns=[])
    
    #print('Traversing file tree...')
    
    glob('*UTC.json')
    
    for file in glob('*UTC.json'):
        with open(file, 'r') as filecontent:
            filename = filecontent.name
            #print('Found JSON file: ' + filename + '. Loading...')
            
            try:
                metadata = orjson.loads(filecontent.read())
            
            except IOError as e:
                #print("I/O Error. Couldn't load file. Trying the next one...")
                continue
            else:
                pass
            #print('Collecting relevant metadata...')
            time = datetime.date.fromtimestamp(int(metadata['node']['taken_at_timestamp']))
            type_ = metadata['node']['__typename']
            likes = metadata['node']['edge_media_preview_like']['count']     
            comments = metadata['node']['edge_media_to_comment']['count']
            total_interaction = metadata['node']['edge_media_preview_like']['count'] + metadata['node']['edge_media_to_comment']['count']
            username = metadata['node']['owner']['username']
            fullname = metadata['node']['owner']['full_name']
            followers = metadata['node']['owner']['edge_followed_by']['count']
            url = metadata['node']["display_url"]
            try:
                text = metadata['node']['edge_media_to_caption']['edges'][0]['node']['text']
            except:
                text = ""
            try:
                video_view_count = metadata['node']['video_view_count']
            except:
                video_view_count = ""
            try:
                post_id = metadata['node']['id']
            except:
                post_id = ""
            minedata = {'post_id' : post_id, 'username' : username, 'fullname':fullname, 'followers' : followers, 'post_type': type_, 'time': time, 
                    'text': text, 'likes': likes, 'comments' : comments, "total_interaction":total_interaction,     'video_view_count':video_view_count,  'url':url}
            #print('Writing to dataframe...')
            dataframe = dataframe.append(minedata, ignore_index=True)
            #print('Closing file...')
            del metadata
            filecontent.close()
    #print('Storing dataframe to CSV file...')
    #print('Done.')
    dataframe['source'] = 'Instagram'
    os.chdir(initial_path)
    return dataframe

In [175]:
list_of_users_data = []
for user in list_of_users:
  df_instagram = parse_instafiles(user, os.getcwd())
  list_of_users_data.append(df_instagram)

C:\Users\faiza\AppData\Local\Temp\ipykernel_3140\1371853461.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(minedata, ignore_index=True)
C:\Users\faiza\AppData\Local\Temp\ipykernel_3140\1371853461.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(minedata, ignore_index=True)
C:\Users\faiza\AppData\Local\Temp\ipykernel_3140\1371853461.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append(minedata, ignore_index=True)
C:\Users\faiza\AppData\Local\Temp\ipykernel_3140\1371853461.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = datafr

In [176]:
full_data = pd.concat(list_of_users_data)

In [177]:
full_data.groupby(by="username").count()["followers"]

username
azuramytha        68
mirandandryana    65
nandaelba         64
notarich_ana      21
prasetyo_sumba    78
reekazees         37
sintabukoi        98
Name: followers, dtype: int64

In [178]:
full_data.to_csv("data/influencers_gorontalo.csv", index=False)

In [179]:
full_data.to_excel("data/influencers_gorontalo.xlsx", index=False)

https://pythonprogramming.medium.com/strategy-analytics-in-python-2-scraping-instagram-30710399cc0b